In [1]:
import os, glob, base64
import datetime
import pandas as pd
import concurrent.futures
import google.generativeai as genai
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from dotenv import find_dotenv, load_dotenv
import json
import yfinance as yf

c:\Users\Max\Desktop\projects\quanticon\pyquant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.options.mode.chained_assignment = None

In [3]:
today_str = datetime.datetime.today().strftime('%Y-%m-%d')

# Configuration
SHEET_ID = "15IfaN1fei9P6BXt0Nj7Rdj7SedDoN_Puzgyb6gUboVQ"
SHEET_NAME = "Sheet1"
DEFAULT_MODEL = "gemini-2.5-flash-preview-05-20"

# Load environment variables from .env file
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

# Check if GEMINI_API_KEY is loaded
if not os.getenv("GEMINI_API_KEY"):
    raise ValueError("GEMINI_API_KEY not found in .env file")

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
def get_trade_ideas():
    today_minus_day = datetime.datetime.today() - datetime.timedelta(1)
    today_minus_day_str = today_minus_day.strftime('%Y-%m-%d')
    print(today_minus_day)

    f_name1 = f'./outputs/{today_str}/backtest_ideas.csv'
    f_name2 = f'./outputs/{today_minus_day_str}/backtest_ideas.csv'
    if os.path.isfile(f_name1): 
        ideas = pd.read_csv(f_name1)
        # return ideas
    elif os.path.isfile(f_name2):
        ideas = pd.read_csv(f_name2)
        # return ideas
    else:
        try:
            scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
            print("setting creds...")
            creds = ServiceAccountCredentials.from_json_keyfile_name(
                os.getenv("GOOGLE_SHEET_API_KEY"), scope
            )
            print('authorizing...')
            client = gspread.authorize(creds)
            print("opening spreadsheet...")
            # sheet = client.open_by_key(SHEET_ID).worksheet(SHEET_NAME)
            spreadsheet = client.open_by_key(SHEET_ID)
            print("opening sheet...")

            sheet = spreadsheet.worksheet(SHEET_NAME)
            sheet_values = sheet.get_all_values()
            ideas_raw = pd.DataFrame(sheet_values[1:], columns=sheet_values[0])
            ideas_raw['date'] = pd.to_datetime(ideas_raw['date'])
            ideas = ideas_raw.loc[ideas_raw['date']<today_minus_day]
            ideas = ideas.loc[(ideas['date of outcome'].isna())|(ideas['date of outcome']=='')|(ideas['date of outcome']=='ongoing')]
            # print(ideas)
            ideas.to_csv(f'./outputs/{today_str}/backtest_ideas.csv', index=False)
        except Exception as e:
            print(f'Error getting sheet: {e}')
            ideas = pd.DataFrame()
    
    return ideas

def extract_last_60_days(ticker):
    ticker_data = yf.download(ticker, period='1y', interval='1d', group_by='ticker')
    df = ticker_data.tail(60)
    return df

def csv_to_base64(df:pd.DataFrame):
    csv_string = df.to_string()
    return csv_string

def call_idea_parse_agent(idea):
    # print('configuring gemini...')
    # api_key = os.getenv("GEMINI_API_KEY")
    # genai.configure(api_key=api_key)
    # print('gemini configured. init gemini...')
    llm = ChatGoogleGenerativeAI(model=DEFAULT_MODEL, temperature=0, api_key=GEMINI_API_KEY)

    system_message = '''You are a helpful text parsing assistant. Your outputs are always in json, and you never provide any extra commentary beyond '''
    system_message += '''what the user requests. It's critical that your ouput is always json, as it's meant to be consumed later by other APIs.'''
    user_message = f"""{idea}\n\nAnalyze this trade idea and parse the ticker (ticker only, no special characters allowed), the entry, """
    user_message += """the stop loss, and the target. Your output should simply be a json object like so: \n\n"""
    user_message += """{{"ticker":<parsed ticker>, "entry":<parsed entry>, "stop loss":<parsed stop loss>, "target":<parsed target>}} \n\n"""
    user_message += """There should be absolutely no other commentary, only the requested information."""
    # print('starting messages')
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", user_message)
    ])

    # print('created messages')
    chain = prompt | llm
    response = chain.invoke({"idea": idea})
    # print('invoked chain')
    return response.content

def call_analysis_agent(idea, date):

    idea_obj_raw = call_idea_parse_agent(idea)
    idea_obj = idea_obj_raw.replace('```json','').replace('```','')
    json_idea = json.loads(idea_obj)

    price_history = extract_last_60_days(json_idea['ticker'])
    prices = price_history[json_idea['ticker']].reset_index()
    # print(prices)
    recent_prices = prices.loc[prices['Date']>date]
    recent_prices['entry'] = float(json_idea['entry'])
    recent_prices['sl'] = float(json_idea['stop loss'])
    recent_prices['tp'] = float(json_idea['target'])
    recent_prices['direction'] = 'bull' if json_idea['target'] > json_idea['entry'] else 'bear'
    
    # entries
    recent_prices['trade_entered'] = 0
    ef1 = recent_prices['direction'] == 'bull'
    ef2 = recent_prices['High'] >= recent_prices['entry']
    ef3 = recent_prices['direction'] == 'bear'
    ef4 = recent_prices['Low'] <= recent_prices['entry']
    recent_prices['trade_entered'] = recent_prices['trade_entered'].mask(((ef1)&(ef2))|((ef3)&(ef4)),1)
    
    # SLs
    recent_prices['sl_hit'] = 0
    sf1 = recent_prices['direction'] == 'bull'
    sf2 = recent_prices['Low'] <= recent_prices['sl']
    sf3 = recent_prices['direction'] == 'bear'
    sf4 = recent_prices['High'] >= recent_prices['sl']
    recent_prices['sl_hit'] = recent_prices['sl_hit'].mask(((sf1)&(sf2))|((sf3)&(sf4)),1)

    # TPs
    recent_prices['tp_hit'] = 0
    tf1 = recent_prices['direction'] == 'bull'
    tf2 = recent_prices['High'] >= recent_prices['tp']
    tf3 = recent_prices['direction'] == 'bear'
    tf4 = recent_prices['Low'] <= recent_prices['tp']
    recent_prices['tp_hit'] = recent_prices['tp_hit'].mask(((tf1)&(tf2))|((tf3)&(tf4)),1)

    analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})

    # price_history_str = csv_to_base64(analysis_gb)

    # llm = ChatGoogleGenerativeAI(model=DEFAULT_MODEL, temperature=0, api_key=GEMINI_API_KEY)

    # system_message = '''You are a helpful stock backtest analysis assistant. Your outputs are always in json, and you never provide any extra commentary beyond '''
    # system_message += '''what the user requests. It's critical that your ouput is always json, as it's meant to be consumed later by other APIs.'''
    # user_message = f"""<idea>{json_idea}</idea><price summary>{price_history_str}</price summary>\n\n"""
    # user_message = f"""Analyze this trade idea, along with the pricing summary, and tell me if 1 of the following occurred: - TP was hit\n- SL """
    # user_message += """was hit\n- the trade is still going\n- never entered trade\n\nAlso, please tell me the date that this occurred on."""
    # user_message += """There should be absolutely no other commentary, only the requested information."""
    # user_message += """Your output should be in the following json format: \n\n"""
    # user_message += """{{"outcome":<simple response>, "date of outcome": <YYYY-MM-DD for date outcome occurred>}} """
    # # print('starting messages')
    # prompt = ChatPromptTemplate.from_messages([
    #     ("system", system_message),
    #     ("user", user_message)
    # ])

    # # print('created messages')
    # chain = prompt | llm
    # response = chain.invoke({"price_history_str": price_history_str, "json_idea":json.dumps(json_idea)})
    # # print('invoked chain')
    # return response.content
    return analysis_gb, json_idea['ticker']

def process_raw_analysis(analysis_raw):
    try:
        trades2 = analysis_raw.reset_index()
        trades3 = trades2.loc[trades2['trade_entered']==1]
        trades3 = trades3.sort_values(['ticker', 'Date'])
        trades4 = trades3.loc[(trades3['sl_hit']>0)|(trades3['tp_hit']>0)]
        trades4['outcome'] = None
        trades4['outcome'] = trades4['outcome'].mask(trades4['tp_hit']==1,'TP Hit')
        trades4['outcome'] = trades4['outcome'].mask(trades4['sl_hit']==1,'SL Hit')
        trades5 = trades4.groupby(['ticker', 'outcome'])['Date'].min().reset_index()
        trades5 = trades5.sort_values(['ticker', 'Date'])
        # completed trades
        trades5 = trades5.drop_duplicates('ticker')
    except:
        trades5 = pd.DataFrame()
    try:
        # ongoing trades
        trades6 = trades3.loc[(trades3['sl_hit']==0)&(trades3['tp_hit']==0)&(~trades3['ticker'].isin(trades5['ticker']))]
    except:
        trades6 = pd.DataFrame()

    trades = pd.concat([trades5,trades6])
    return trades

def update_google_sheet(date_str, play_text):
    return

def process_ideas(df):
    # ai_output = ""  # Initialize ai_output
    # date_str = datetime.datetime.now().strftime("%Y-%m-%d")
    # failure_step = 'last 60'

    # try:
    #     df_last60 = extract_last_60_days(file_path)
        
    #     failure_step = 'csv conversion'
    #     base64_data = csv_to_base64(df_last60)
        
    #     failure_step = 'call agent'
    #     ai_output = call_agent(base64_data)
        
    #     failure_step = 'append to sheet'
    #     append_to_google_sheet(date_str, ai_output)
    #     print(f"Successfully processed {os.path.basename(file_path)}")
    #     return ai_output  # No error
    # except Exception as e:
    #     error_message = f"Error processing {os.path.basename(file_path)} on {date_str}: {e}\nAI Output: {ai_output}\n\n"
    #     print(error_message)
    #     print(f'step failed: {failure_step}')
    #     print('=====')
    #     return error_message
    return

In [5]:
ideas = get_trade_ideas()
ideas

2025-08-15 19:30:51.132851
setting creds...
authorizing...
opening spreadsheet...
opening sheet...


,date,play,date of outcome,outcome
63,2025-07-11,Ticker: $FLR\nTrade Type: Swing\nBias: Bullish...,,
108,2025-07-26,Ticker: $MRVL\nTrade Type: Swing\nBias: Bullis...,,
109,2025-07-26,Ticker: $KNX\nTrade Type: Scalp\nBias: Bearish...,,
110,2025-07-26,Ticker: $IOSP\nTrade Type: Swing\nBias: Bearis...,,
111,2025-07-26,Ticker: $DIN\nTrade Type: Scalp\nBias: Bearish...,,
112,2025-07-26,Ticker: $ENVX\nTrade Type: Scalp\nBias: Bullis...,,
113,2025-07-26,Ticker: $CCS\nTrade Type: Scalp\nBias: Bearish...,,
114,2025-07-26,Ticker: $ATRO\nTrade Type: Swing\nBias: Bullis...,,
115,2025-07-26,Ticker: $ALNT\nTrade Type: Swing\nBias: Bullis...,,
116,2025-07-26,Ticker: $BXC\nTrade Type: Scalp\nBias: Bullish...,,


In [ ]:
import time

trades = pd.DataFrame()

for (i, row) in ideas.iterrows():
    # print(i)
    # print(row['play'])
    # idea_obj_raw = call_idea_parse_agent(row['play'])
    # idea_obj = idea_obj_raw.replace('```json','').replace('```','')
    # json_idea = json.loads(idea_obj)
    # # print(json_idea, json_idea['ticker'])
    # price_history = extract_last_60_days(json_idea['ticker'])
    # price_history_str = csv_to_base64(price_history)

    # trades.append([row['date'],json_idea,price_history_str])
    print(row['date'])
    print(row['play'])
    analysis_raw, ticker = call_analysis_agent(row['play'], row['date'])
    analysis_raw['ticker'] = ticker
    # analysis_gb = analysis_raw.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})
    
    print(analysis_raw)
    analysis_final = process_raw_analysis(analysis_raw)
    send_date = analysis_final.loc[analysis_final['ticker']==ticker, ['Date']]
    send_outcome = analysis_final.loc[analysis_final['ticker']==ticker, ['outcome']]
    new_row_data = [row['date'], row['play'], send_date, send_outcome]
    trades = pd.concat([trades, analysis_raw])
    # analysis_obj = analysis_raw.replace('```json','').replace('```','')
    # json_analysis = json.loads(analysis_obj)
    # print(json_analysis)
    # trades.append([row['date'],row['play'],json_analysis['date of outcome'],json_analysis['outcome']])
    time.sleep(2)

trades

2025-07-11 00:00:00
Ticker: $FLR
Trade Type: Swing
Bias: Bullish
Entry: $53.70
Stop-Loss: $50.50
Target: $58.00
Option Contract: $55.00 Call August 2025
Rationale: FLR is in a strong, confirmed uptrend with higher highs and higher lows, supported by positive EMA trends and bullish indicator flags. Recent price action shows consolidation after a significant move, suggesting a potential continuation of the uptrend upon breaking recent resistance.


[*********************100%***********************]  1 of 1 completed
C:\Users\Max\AppData\Local\Temp\ipykernel_17380\3711019278.py:115: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})


Price       Date       Open       High        Low      Close    Volume
0     2025-05-21  37.950001  38.169998  37.340000  37.880001   2530800
1     2025-05-22  37.619999  38.189999  37.480000  37.580002   2560000
2     2025-05-23  37.660000  41.330002  37.660000  41.040001   8254000
3     2025-05-27  42.250000  43.860001  41.419998  43.540001   9138800
4     2025-05-28  43.549999  43.740002  42.080002  42.439999   5086000
5     2025-05-29  42.799999  43.080002  41.189999  42.160000   4645700
6     2025-05-30  41.959999  42.240002  41.070000  41.580002   2566000
7     2025-06-02  41.570000  41.580002  40.410000  41.150002   3078800
8     2025-06-03  42.490002  43.689999  42.090000  43.389999   5073700
9     2025-06-04  43.200001  43.759998  42.759998  42.889999   2938800
10    2025-06-05  42.959999  43.310001  42.380001  42.950001   2310500
11    2025-06-06  43.470001  44.560001  43.169998  44.430000   4377500
12    2025-06-09  45.330002  46.299999  45.080002  45.139999   4855900
13    

[*********************100%***********************]  1 of 1 completed
C:\Users\Max\AppData\Local\Temp\ipykernel_17380\3711019278.py:115: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})


Price       Date       Open       High        Low      Close    Volume
0     2025-05-21  60.900150  62.398923  59.631188  60.060837  14022800
1     2025-05-22  59.361409  62.808588  59.271482  61.799412  16653100
2     2025-05-23  60.100806  61.249863  59.950927  60.640362  17443200
3     2025-05-27  62.888522  65.466410  62.408915  63.767803  25511700
4     2025-05-28  64.747005  65.846104  64.447248  64.537170  23421200
5     2025-05-29  67.304908  67.534715  63.278205  63.677876  38742400
6     2025-05-30  60.750272  60.930125  58.562064  60.140770  48347500
7     2025-06-02  59.701129  62.249043  59.481308  61.419724  18270300
8     2025-06-03  61.979266  63.458055  61.099989  62.308998  19286600
9     2025-06-04  62.988438  66.975172  61.809403  66.245773  26990200
10    2025-06-05  67.214980  67.534718  64.756992  65.106712  23426600
11    2025-06-06  66.915229  69.722929  66.255765  68.294098  22926300
12    2025-06-09  69.413182  71.181731  68.903597  69.083450  19351800
13    

...

[*********************100%***********************]  1 of 1 completed
C:\Users\Max\AppData\Local\Temp\ipykernel_17380\3711019278.py:115: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})


Price       Date       Open       High        Low      Close   Volume
0     2025-05-21  54.240002  54.480000  51.419998  51.459999   494500
1     2025-05-22  51.080002  53.230000  51.080002  52.990002   531200
2     2025-05-23  51.349998  52.560001  51.000000  52.340000   247200
3     2025-05-27  53.380001  55.900002  53.040001  55.889999   351400
4     2025-05-28  56.750000  57.139999  55.709999  56.509998   412000
5     2025-05-29  57.500000  57.970001  55.400002  55.689999   409800
6     2025-05-30  55.220001  56.560001  54.869999  55.580002   445600
7     2025-06-02  55.610001  56.049999  53.990002  55.150002   392900
8     2025-06-03  55.500000  58.000000  54.740002  57.459999   510200
9     2025-06-04  57.610001  58.230000  56.380001  56.380001   294500
10    2025-06-05  55.980000  57.049999  54.439999  54.910000   236500
11    2025-06-06  56.259998  58.349998  55.689999  58.250000   312300
12    2025-06-09  59.099998  59.810001  56.180000  57.169998   414900
13    2025-06-10  57

[*********************100%***********************]  1 of 1 completed
C:\Users\Max\AppData\Local\Temp\ipykernel_17380\3711019278.py:115: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})


Price       Date       Open       High        Low      Close    Volume
0     2025-05-21  18.490000  19.218000  16.799999  17.070000   3621900
1     2025-05-22  17.080000  18.090000  17.080000  17.610001   1972600
2     2025-05-23  16.799999  18.039000  16.750000  17.879999   2237500
3     2025-05-27  18.500000  19.198999  17.889999  18.559999   2739300
4     2025-05-28  18.670000  18.959999  17.565001  17.650000   2913800
5     2025-05-29  18.240000  18.290001  16.530001  16.930000   2613000
6     2025-05-30  16.559999  16.700001  15.060000  15.420000   5025600
7     2025-06-02  15.310000  15.770000  15.060000  15.515000   1896400
8     2025-06-03  15.900000  17.209999  15.470000  16.760000   3519200
9     2025-06-04  17.430000  17.614000  16.530001  17.110001   2137100
10    2025-06-05  17.240000  17.434999  16.000000  16.260000   3057400
11    2025-06-06  16.565001  17.450001  16.260000  16.830000   2508100
12    2025-06-09  17.250000  17.250000  16.600000  16.870001   2281800
13    

Price                             Date ticker
trade_entered sl_hit tp_hit                  
0             0      0      2025-07-16    FLR
              1      0      2025-08-01    FLR
1             0      0      2025-07-14    FLR
                     0      2025-07-28   MRVL
                     1      2025-07-29   MRVL
                     0      2025-07-28    KNX
                     1      2025-07-30    KNX
0             0      0      2025-07-28   IOSP
1             0      0      2025-07-29   IOSP
                     1      2025-07-31   IOSP
                     0      2025-07-28    DIN
                     1      2025-07-29    DIN
0             1      0      2025-07-30   ENVX
1             0      0      2025-07-28   ENVX
0             1      0      2025-08-05    CCS
1             0      0      2025-07-28    CCS
                     1      2025-07-30    CCS
0             0      0      2025-07-28   ATRO
              1      0      2025-08-07   ATRO
1             0      0      2025-07-30   ATRO
0             0      0      2025-07-28   ALNT
              1      0      2025-08-01   ALNT
1             0      0      2025-07-29   ALNT
                     1      2025-08-07   ALNT
              1      0      2025-08-04   ALNT
0             0      0      2025-07-28    BXC
              1      0      2025-07-29    BXC
              0      0      2025-08-04    ALK
              1      0      2025-07-28    ALK
1             0      0      2025-08-01    ALK
0             1      0      2025-08-01   ACMR
1             0      0      2025-07-28   ACMR
                     1      2025-07-29   ACMR
0             0      0      2025-08-07   MLTX
1             0      0      2025-08-04   MLTX
0             1      0      2025-08-04   AAMI
1             0      0      2025-08-11    WDC
0             1      0      2025-08-12   XPEL
1             0      0      2025-08-11   XPEL
0             1      0      2025-08-11    HZO
                     0      2025-08-11     EL
1             0      0      2025-08-13     EL
                     0      2025-08-11    CPS
                     0      2025-08-11   CDTX
                     1      2025-08-14   CDTX
0             1      0      2025-08-13    BXC
1             0      0      2025-08-11    BXC
              1      0      2025-08-12    BXC
              0      0      2025-08-11   CAVA
                     1      2025-08-13   CAVA
0             1      0      2025-08-11   ALSN
1             1      0      2025-08-15   ALSN
0             1      0      2025-08-12   ALGT
1             0      0      2025-08-11   ALGT
0             1      0      2025-08-13   AAOI
1             0      1      2025-08-11   AAOI
              1      0      2025-08-12   AAOI

In [7]:
trades2 = trades.reset_index()
trades3 = trades2.loc[trades2['trade_entered']==1]
trades3 = trades3.sort_values(['ticker', 'Date'])
trades3

Price,trade_entered,sl_hit,tp_hit,Date,ticker
55,1,0,1,2025-08-11,AAOI
56,1,1,0,2025-08-12,AAOI
31,1,0,0,2025-07-28,ACMR
32,1,0,1,2025-07-29,ACMR
53,1,0,0,2025-08-11,ALGT
29,1,0,0,2025-08-01,ALK
22,1,0,0,2025-07-29,ALNT
24,1,1,0,2025-08-04,ALNT
23,1,0,1,2025-08-07,ALNT
51,1,1,0,2025-08-15,ALSN


In [ ]:
# completed trades
trades4 = trades3.loc[(trades3['sl_hit']>0)|(trades3['tp_hit']>0)]
trades4['outcome'] = None
trades4['outcome'] = trades4['outcome'].mask(trades4['tp_hit']==1,'TP Hit')
trades4['outcome'] = trades4['outcome'].mask(trades4['sl_hit']==1,'SL Hit')
trades5 = trades4.groupby(['ticker', 'outcome'])['Date'].min().reset_index()
trades5 = trades5.sort_values(['ticker', 'Date'])
trades5 = trades5.drop_duplicates('ticker')
trades5

,ticker,outcome,Date
1,AAOI,TP Hit,2025-08-11
2,ACMR,TP Hit,2025-07-29
3,ALNT,SL Hit,2025-08-04
5,ALSN,SL Hit,2025-08-15
6,BXC,SL Hit,2025-08-12
7,CAVA,TP Hit,2025-08-13
8,CCS,TP Hit,2025-07-30
9,CDTX,TP Hit,2025-08-14
10,DIN,TP Hit,2025-07-29
11,IOSP,TP Hit,2025-07-31


In [ ]:
# entered trades that are still ongoing
trades3.loc[(trades3['sl_hit']==0)&(trades3['tp_hit']==0)&(~trades3['ticker'].isin(trades5['ticker']))]

Price,trade_entered,sl_hit,tp_hit,Date,ticker
53,1,0,0,2025-08-11,ALGT
29,1,0,0,2025-08-01,ALK
19,1,0,0,2025-07-30,ATRO
42,1,0,0,2025-08-11,CPS
41,1,0,0,2025-08-13,EL
13,1,0,0,2025-07-28,ENVX
2,1,0,0,2025-07-14,FLR
34,1,0,0,2025-08-04,MLTX
36,1,0,0,2025-08-11,WDC
38,1,0,0,2025-08-11,XPEL
